# Tuning Porte-Agel Parameters

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import foxes
import foxes.variables as FV

import sys
sys.path.append( '/home/sengers/code/python/')
from util import *

dir_edwork = '/home/sengers/eddy/work/palm/current_version/JOBS/'
from scipy.optimize import least_squares

In [2]:
def sdata_shear(z, hh=90,WS_hh=8.0,WD_hh=270.,shear=0.2,veer=0):
    """ 
    z = heights [m]
    hh = hub height [m]
    WS_hh = wind speed at hub height [m/s]
    WD_hh = wind direction at hub height [deg]
    shear = shear exponent [-]
    veer = veer (directional shear) [deg/m]
    """
    z = np.asarray(z)
    WS = WS_hh*(z/hh)**shear
    WD = WD_hh+veer*(z-hh)
    columns =  ['WS-'+str(k) for k in z] +  ['WD-'+str(k)for k in z]
    sdata = pd.DataFrame(list(WS)+list(WD),index=columns,columns=[0])
    return sdata.T

def yz_plane(algo, xs, y, z):
    import foxes.constants as FC
    from foxes.utils import wd2wdvec, wd2uv
    farm_results = algo.calc_farm()
    # prepare:
    n_x = np.append(wd2wdvec(270-180), [0.0], axis=0) ## -180 to get [1,0,0]
    n_z = np.array([0.0, 0.0, 1.0])
    n_y = np.cross(n_z, n_x)

    # project to axes:
    # xyz = np.zeros((n_states, n_turbines, 3), dtype=FC.DTYPE)
    xyz = np.zeros((1,1,3), dtype=FC.DTYPE) ## (n_states, n_turbines, n_dimensions)
    xyz[:, :, 0] = farm_results[FV.X]
    xyz[:, :, 1] = farm_results[FV.Y]
    xyz[:, :, 2] = farm_results[FV.H]
    xx = np.einsum("std,d->st", xyz, n_x)
    yy = np.einsum("std,d->st", xyz, n_y)
    zz = np.einsum("std,d->st", xyz, n_z)
    del xyz

    y_pos, y_res = np.linspace(
        np.min(y),
        np.max(y),
        num=int((np.max(y) - np.min(y)) / 5) + 1,
        endpoint=True,
        retstep=True,
        dtype=None,
    )
    z_pos, z_res = np.linspace(
        np.min(z),
        np.max(z),
        num=int((np.max(z) - np.min(z)) / 5) + 1,
        endpoint=True,
        retstep=True,
        dtype=None,
    )
    
    N_y, N_z = len(y_pos), len(z_pos)
    n_pts = len(y_pos) * len(z_pos)
    data_all = []
    for x in xs:
        x_pos = x
        g_pts = np.zeros((1, N_y, N_z, 3), dtype=FC.DTYPE)
        g_pts[:] += x_pos * n_x[None, None, None, :]
        g_pts[:] += y_pos[None, :, None, None] * n_y[None, None, None, :]
        g_pts[:] += z_pos[None, None, :, None] * n_z[None, None, None, :]
        g_pts = g_pts.reshape(1, n_pts, 3)

        point_results = algo.calc_points(farm_results, points=g_pts)
        data = point_results[FV.WS].values
        data = data[0].reshape([N_y, N_z]).T ## reshape
        data_all += [data]
    return np.asarray(data_all)

In [4]:
## 
class tuningProcess():
    """
This class includes one tuning methods:
    
f_pwr(...) : minimize difference in available power in the rotor plane 
            at several downstream distances.
"""  
    def __init__(self,df_in,cases,dDs):
        self.cases = cases
        self.df = df_in
        self.dDs = np.asarray(dDs)

        u_refs = []
        for i,case in enumerate(cases):
            file = dir_edwork+case+'/OUTPUT/'+case+'_masked_M02_avg60min.nc'
            tmp = []
            for dD in dDs:
                u = np.asarray(get_var(file,'u'))[0,:,:,dD]
                if i == 0:
                    y = np.asarray(get_var(file,'y'))-1280
                    z = np.asarray(get_var(file,'z'))
                    mask = mask_turb(y,z,rcy=0,hh=90)
                u[mask==0]=np.nan
                tmp += [np.nanmean(u)]
            u_refs += [tmp]
        self.mask = np.asarray(mask)
        self.u_refs = np.asarray(u_refs)
        self.y = np.asarray(y)
        self.z = np.asarray(z)

    def tuneWM(self):
        """
        xx : tuning model parameters.

        Returns
        -------
        Array
            optimalized tuning model parameters.

        """
        def f_pwr(xx):
            u_wm = []
            for case in cases:
                ## Initialize
                yawm = np.asarray(self.df.loc[case,'yawm'])
                sdata = sdata_shear(self.z, hh=90,WS_hh=self.df.loc[case,'WS'],WD_hh=270,shear=self.df.loc[case,'shear'],veer=0)

                ## states
                states = foxes.input.states.MultiHeightTimeseries(
                    data_source=sdata,
                    heights=self.z,
                    output_vars=[FV.WS, FV.WD, FV.TI, FV.RHO],
                    fixed_vars={FV.RHO: 1.225, FV.TI: self.df.loc[case,'TI']/100},
                )
                ## Build and execute
                mbook = foxes.ModelBook()
                mbook.turbine_types["NREL5"] = foxes.models.turbine_types.PCtFile(
                    "NREL-5MW-D126-H90.csv"
                )
                mbook.turbine_models["set_yawm"] = foxes.models.turbine_models.SetFarmVars()
                mbook.turbine_models["set_yawm"].add_var(FV.YAWM, yawm)

                # mbook.turbine_models["k_tuning"] = foxes.models.turbine_models.kTI(kTI=xx[2],kb=xx[3])
                # mbook.print_toc(subset="turbine_models", search="")                

                farm = foxes.WindFarm()
                farm.add_turbine(
                    foxes.Turbine(
                        xy=[0.0, 0.0],
                        turbine_models=["set_yawm", "yawm2yaw", "NREL5", "kTI_05"],
                    )
                )
                mbook.wake_frames["yawed_tuning"] = foxes.models.wake_frames.YawedWakes(
                    alpha=xx[0],beta=xx[1]) ## add new wake_frame (deflection model)


                algo = foxes.algorithms.Downwind(
                    mbook,
                    farm,
                    states,
                    rotor_model="centre",
                    wake_models=["PorteAgel_linear", "CrespoHernandez_max"],
                    wake_frame="yawed_tuning", ## use new wake_frame
                    partial_wakes_model="auto",
                    chunks=None,
                    verbosity=0,
                )
                
                farm_results = algo.calc_farm()
                data_all = yz_plane(algo, self.dDs*126, self.y, self.z)
                u_wm_tmp = []
                for data in data_all:
                    data[self.mask==0]=np.nan
                    # plt.contourf(data)
                    # plt.colorbar()
                    # plt.show()
                    u_wm_tmp += [np.nanmean(data)]
                u_wm += [u_wm_tmp]
            print('u_wm',u_wm)
            print('u_refs',self.u_refs)
            Err = np.sum(np.abs(np.asarray(u_wm)**3-self.u_refs**3))
            print(xx,Err)
            print()
            return Err


        
        opt_tune    = least_squares(f_pwr,[0.8,0.2,0.5,0.1],max_nfev=1500)
        print(opt_tune.x)

        return opt_tune.x

            

## Prep
cases = ['NBL_00_ADMR','NBL_30_ADMR','NBL_-30_ADMR']
df = load_csv('/home/sengers/eddy/work/analysis/pickles/ALLsmooth_parameters_avg60min.csv')
df.index = [k[:-1] for k in df.index]
df = df.T[cases].T
df = df[['yaw','F_hh','shear','veer','TIamb']]
df.columns = ['yawm','WS','shear','veer','TI']

## Run
tuning      = tuningProcess(cases=cases,df_in=df,dDs=range(4,10))
pars        = tuning.tuneWM()


# cases = ['NBL_00_ADMR']
# df = load_csv('/home/sengers/eddy/work/analysis/pickles/ALLsmooth_parameters_avg60min.csv')
# df.index = [k[:-1] for k in df.index]
# df = df.T[cases].T
# df = df[['yaw','F_hh','shear','veer','TIamb']]
# df.columns = ['yawm','WS','shear','veer','TI']

# ## Run
# tuning      = tuningProcess(cases=cases,df_in=df,dDs=[4])
# pars        = tuning.tuneWM()

       

/home/sengers/code/python/util.py:171: RuntimeWarning: invalid value encountered in sqrt
  y_left = rcy - np.sqrt(rr**2-(z_tmp-hh)**2) ## use equation of circle
/home/sengers/code/python/util.py:172: RuntimeWarning: invalid value encountered in sqrt
  y_right = rcy + np.sqrt(rr**2-(z_tmp-hh)**2) ## to determine left,right edge


Turbine 0, T0: set_yawm, yawm2yaw, NREL5, kTI_05
Turbine 0, T0: set_yawm, yawm2yaw, NREL5, kTI_05
Turbine 0, T0: set_yawm, yawm2yaw, NREL5, kTI_05
u_wm [[6.543471378422324, 6.837817396604123, 7.058978732298599, 7.230391202141002, 7.366206389880533, 7.47569682012965], [6.929065950234566, 7.17001342370107, 7.34554041246193, 7.479170870387736, 7.5842263483929715, 7.6688405260985055], [6.90017300833629, 7.142461437175501, 7.319135046274181, 7.453710070798832, 7.559528341305022, 7.644753085424613]]
u_refs [[5.39896769 5.82335445 6.16247756 6.41903254 6.6202741  6.7813668 ]
 [6.48669593 6.76625341 6.9719514  7.11675343 7.22132538 7.30936018]
 [6.65343377 6.97587918 7.21692477 7.39076638 7.52476281 7.62767625]]
[0.8 0.2 0.5 0.1] 1140.2548563988378

Turbine 0, T0: set_yawm, yawm2yaw, NREL5, kTI_05
Turbine 0, T0: set_yawm, yawm2yaw, NREL5, kTI_05
Turbine 0, T0: set_yawm, yawm2yaw, NREL5, kTI_05
u_wm [[6.543471385547382, 6.8378174018539895, 7.058978736315912, 7.2303912052937225, 7.36620639240255